### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


In [3]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [4]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [5]:
head(df,2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,100,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
df$flag <- FALSE
df$true_false <- FALSE


In [7]:
head(df, 2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.0700,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


## <font color = yellow>1. Create data & dfs

In [8]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

create_data()


In [9]:
dim(df)


[1] 41 23

## <font color = lime> Examining data and dfs

In [10]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 2)
head(sells,2)

[1] 23

[1] 18

[1] 18 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.51240,5512.40,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [11]:
dim(b)
b

[1]  1 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


### <font color = yellow>2. update_b() - updates the basic b variable with data from buy/sell rows 
    
#### b>s, b<s, b==s,

In [12]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


In [13]:
last_row <- tail(buys, n = 1)
last_row

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
39,39,2023-05-17,Buy,AMC,1000,5.009,5009,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [14]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE         
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {            
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
          
 #           s$type <- "SOLD"            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [15]:
dim(buys)
dim(sells)

[1] 23 23

[1] 18 23

### <font color = yellow>3. Update the sales_records (rbind) with updated_b 

In [16]:
# sale_records <- data.frame()
# sale_records <- rbind(sale_records, updated_b)
# sale_records
                      

In [17]:
df_name <- "iris"  # Replace with your data frame name
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

[1] TRUE

In [18]:
df_name <- "sale_records"
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

[1] FALSE

In [19]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    cat("Newly created sale_records with updated_b data. ", "\n")
    sale_records <- data.frame()
    sale_records <- rbind(sale_records, updated_b)    
}
sale_records

Newly created sale_records with updated_b data.  


index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [20]:
updated_b

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [21]:
dim(buys)
dim(sells)

[1] 23 23

[1] 18 23

### <font color = yellow> 4. with_c/f, update buys or sells with a new row (add_row) when c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


#### <font color = lime> How it's done

In [22]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


In [23]:
first_row <- head(sells, 1)
first_row

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [24]:
last_row <- tail(sale_records, n = 1)
last_row

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [25]:
dim(buys)
dim(sells)

[1] 23 23

[1] 18 23

In [26]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_withcf <- function(buys, sells) {

    #  Check the most recent row ??? of sale_record or the last row of sale_record (was bind() or add_row() ? )
    last_row <- tail(sale_records, n = 1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    
    #  b > s  ... buy c/f will be created 
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
        b1 <- last_row
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- FALSE
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test

    #   s > b ... sell carryforward will be created    create s1
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
       
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
           # cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/f"
        s1 <- last_row
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <- s1$sold_cf_pps
        s1$basis <- s1$sold_cf_basis
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
                      
        }

    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ], dim of buys increases with c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... dim of sells increases with c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_withcf. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
return_list <- update_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)



 There is Sold carryforward. Update sells  900 
printing: s1 ... dim of sells increases with c/f  


[1] 23 23

[1] 19 23

In [27]:
buys[1:2, ]
sells[1:2,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.40,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = yellow>5. Snip(), delete row from buys and sells / replace row when c/f 

In [28]:
buys[1:2,]
sells[1:2,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.40,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


In [29]:
range(x <- sort(round(stats::rnorm(10) - 1.2, 1)))

if(any(x < 0)) cat("x contains negative values\n")

[1] -3.4  0.3

x contains negative values


In [30]:
if(any(buys[1,] == TRUE)) cat("buys contains cf \n")

buys contains cf 


In [31]:

buys[1,]
sells[1,]
dim(buys)
dim(sells)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 23 23

[1] 19 23

In [32]:
cat("buys$type[1] is ", buys$type[1], "\n")
sells$type[1]

buys$type[1] is  Buy 


[1] "Sell CF"

In [33]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    
   
    # snip .... if 
# if(any(buys$type[1] == "Buy CF") ||  any(sells$type[1] == TRUE))  {         
    if(buys$type[1] == "Buy CF")   {         
        cat("buys contains carryforward - Replace buys and DO snip sells \n") 
        
         buys <- buys[-2, ]
        # buys <-  buys %>% add_row(b1, .before = 1)
        sells <- sells[-1,]       
        return(list(b1, s1, buys, sells))    
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))    
    } else {
        cat("There is no carryforward in buys or sells, DO snip buys and Do snip sells \n")
        buys <- buys[-1,]
        sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
}    
   

# fct call passing 2 arguments
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

sells contains carryforward .. Replace sells and DO snip buys 


[1] 22 23

[1] 18 23

In [34]:
buys[1,]
sells[1,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop1 review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells

In [35]:
head(buys,3)
head(sells,3)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.6250,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
5,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
7,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.51240,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop 1 review sale_record for accuracy

In [36]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [37]:
# stopps #  Old code

In [38]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} 

sale_records exists:      


In [39]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


## <font color = yellow> Proof of Concept above. Make it loop 2 x.

#### <font color = yellow>1. Create data & dfs

In [40]:
b <- buys[1,]
s <- sells[1,]

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [41]:
updated_b <- update_b()
updated_b  
    

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.625,562.5,FALSE


### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [42]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}


sale_records exists:      


### The sale_record is correct for buy c/f in the second loop

### the idea is for the second loop to only process the sale of 50 when buy cf exist, t/f 

###  at the end of loop two, the buys should be showing the next loop is ready for 1000 (next buys)

### and the sells s/b showing 950

In [43]:
sale_records

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,407,100,FALSE,0,⋯,4.070,407.0,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.6250,562.50,FALSE


#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [44]:
# fct call
return_list <- update_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)



 There is Buy carryforward. Update buys  100 
printing updated buys[1 ], dim of buys increases with c/f  


[1] 23 23

[1] 18 23

## This is the place the issue becomes apparent

In [45]:
#  These are the two values in this transaction

head(buys,3)
head(sells, 3)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,2023-01-20,Buy CF,c/f,100,5.625,562.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
2,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,5,2023-01-23,Buy,AMC,1000,5.788,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Sell CF,c/f,900,5.51240,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [46]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    
   
    # snip .... if 
# if(any(buys$type[1] == "Buy CF") ||  any(sells$type[1] == TRUE))  {         
    if(buys$type[1] == "Buy CF")   {         
        cat("buys contains carryforward - remove buys c/f row and DO snip sells \n") 
        
         buys <- buys[-1, ]
        # buys <-  buys %>% add_row(b1, .before = 1)
        sells <- sells[-1,]       
        return(list(b1, s1, buys, sells))    
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))    
    } else {
        cat("There is no carryforward in buys or sells, DO snip buys and Do snip sells \n")
        buys <- buys[-1,]
        sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
}    
   

# fct call passing 2 arguments
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

buys contains carryforward - remove buys c/f row and DO snip sells 


[1] 22 23

[1] 17 23

In [47]:
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

There is no carryforward in buys or sells, DO snip buys and Do snip sells 


#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [48]:
sale_records

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,407,100,FALSE,0,⋯,4.070,407.0,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.6250,562.50,FALSE


In [49]:
dim(buys)
dim(sells)
head(buys,3)
head(sells,3)

[1] 21 23

[1] 16 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,9,2023-02-10,Buy,AMC,2000,4.7450,9490.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,6,2023-02-03,Sell,AMC,1000,6.2550,6255.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,8,2023-02-03,Sell,AMC,1000,6.1901,6190.1,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
6,10,2023-02-21,Sell,AMC,3000,5.8550,17565.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


### The sale_record is correct for buy c/f in the second loop

### the idea is for the second loop to only process the sale of 50 when buy cf exist, t/f 

###  at the end of loop two, the buys should be showing the next loop is ready for 1000 (next buys)

### and the sells s/b showing 950

In [50]:
stopps

ERROR: Error in eval(expr, envir, enclos): object 'stopps' not found


#### <font color = lime> QC: review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells